# Chapter 6 - Regularization

In [1]:
import sys
sys.path.append("../")
from utils import *
np.random.seed(1)

## Regularized Regression Methods

In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge


def fit_model(model_gen, lambdas, X_train, y_train, X_test, y_test):  
    coefs  = pd.DataFrame([], columns=list(X_train.columns),  index = lambdas)
    losses = pd.DataFrame([], columns=["mse", "reg", "loss"], index = lambdas)

    for lam in lambdas:
        model = model_gen(lam, X_train, y_train)
        coefs.loc[lam, :] = model.coef_

        mse = mean_squared_error(y_test, model.predict(X_test))
        reg = lam*np.linalg.norm(model.coef_, ord=1)
        losses.loc[lam, :] = [mse, reg, mse+reg]
    return coefs, losses


models = {"Lasso": lambda lam, x, y: Lasso(alpha=lam, normalize=True, max_iter=10000, tol=1e-4, random_state=3).fit(x, y),
          "Ridge": lambda lam, x, y: Ridge(alpha=lam, normalize=True, random_state=3).fit(x, y)}

lambda_dicts = {"Lasso": 2**np.linspace(-10, .5, 100), "Ridge": 2**np.linspace(-7, 3, 100)}

In [7]:
# Load dataset and split to train and test sets
tr, te= train_test_split(pd.read_csv("../data/mtcars.csv").drop(columns=["model"]).dropna(), test_size=0.4, random_state=5)
X_train, y_train, X_test, y_test = tr.loc[:, tr.columns != "mpg"], tr["mpg"], te.loc[:, te.columns != "mpg"], te["mpg"]


# Load and fit a Lasso or Ridge regression model
learner = "Ridge" #"Lasso"
lambdas = lambda_dicts[learner]
coefs, losses = fit_model(models[learner], lambdas, X_train, y_train, X_test, y_test) 


fig = make_subplots(rows=2, cols=1, subplot_titles=[r'$\text{Regularization Path}$', r'$\text{Model Losses}$'], 
                    row_heights=[400,200], vertical_spacing=.1)

# Plot the regularization path for each feature
for i, col in enumerate(X_train.columns):
    fig.add_trace(go.Scatter(x=lambdas, y=coefs.loc[:, col], mode='lines', name=col, legendgroup="1"))


# Plot the losses graph and mark lambda with lowest loss
lam = np.argmin(losses.loc[:, 'loss'])
fig.add_traces([go.Scatter(x=lambdas, y=losses.loc[:, 'mse'], mode='lines', name="Fidelity Term - MSE", legendgroup="2"),
                go.Scatter(x=lambdas, y=losses.loc[:, 'reg'], mode='lines', name="Regularization Term", legendgroup="2"),
                go.Scatter(x=lambdas, y=losses.loc[:, 'loss'], mode='lines', name="Joint Loss", legendgroup="2"),
                go.Scatter(x=[lambdas[lam]], y=[losses.loc[:, 'loss'].values[lam]], mode='markers', showlegend=False,
                           marker=dict(size=8, symbol="x"), hovertemplate="Lambda: %{x}<extra></extra>")], 2, 1)

fig.update_layout(hovermode='x unified', margin=dict(t=50), 
                  legend = dict(tracegroupgap = 60),
                  yaxis  = dict(title=r"$\text{Coefficients}$"),
                  yaxis2 = dict(title=r"$\text{Test Loss}$"),
                  title  = rf"$\text{{Fitting {learner} Regression}}$")
# Uncomment to plot x-axis in log scale
#fig.update_xaxes(type="log")

fig.write_image(f"../figures/{learner}_fitting.png")
fig.show()

In [4]:
lambdas = 2**np.linspace(-10, .5, 100)
regressors = {"Ridge": fit_model(models["Ridge"], lambdas, X_train, y_train, X_test, y_test),
              "Lasso": fit_model(models["Lasso"], lambdas, X_train, y_train, X_test, y_test)}

coefs = np.array([LinearRegression().fit(X_train, y_train).coef_,
                  regressors["Ridge"][0].iloc[np.argmin(regressors["Lasso"][1].loc[:, "loss"])],
                  regressors["Lasso"][0].iloc[np.argmin(regressors["Lasso"][1].loc[:, "loss"])]])

fig = go.Figure(layout=go.Layout(title=r"$\text{Regression Models Coefficients}$",
                                 width=600, height=400,
                                 xaxis=dict(range=[-1.1, 1.1], showticklabels=False, zeroline=False) ))

fig.add_annotation(x=-.8, y=6.3, text=r"$\text{LS Coefficients}$",   showarrow=False)
fig.add_annotation(x=0,   y=6.3, text=r"$\text{Ridge Coefficients}$", showarrow=False)
fig.add_annotation(x=.8,  y=6.3, text=r"$\text{Lasso Coefficients}$", showarrow=False)

for i, col in enumerate(X_train.columns):
    fig.add_trace(go.Scatter(x=[-1, 0, 1], y=coefs[:, i], mode='markers+lines', name=col, line={'dash':'dot'}))

fig.write_image(f"../figures/regression_model_coefficients.png")
fig.show()